In [67]:
import random 
from random import randint

### Exponensiación rapida

http://www-users.math.umn.edu/~garrett/crypto/Code/FastPow_Python.html 

In [70]:
def exp_mod(a,b,n):
    X = a
    E = b
    Y = 1
    while E > 0:
        if E % 2 == 0:
            X = (X * X) % n
            E = E/2
        else:
            Y = (X * Y) % n
            E = E - 1
    return Y

In [71]:
def exp(a,b):
    if b == 0:
        return 1
    else:   
        result = a
        for i in range(b-1):
            result *=a
        return result
        

### Miller Rabin

#### Referencias:

https://gist.github.com/Ayrx/5884790 

https://www.geeksforgeeks.org/primality-test-set-3-miller-rabin/ 

https://github.com/womogenes/Miller-Rabin/blob/master/miller_rabin.py

La función power se implemento para mejorar la velocidad de computo del numero primo. Sin embargo, esta usa shift y AND, en caso de no poner utilizarlos esta funcion puede ser remplazada por exp_mod

In [78]:
def power(x, y, p):
     
    # Initialize result
    res = 1;
     
    # Update x if it is more than or
    # equal to p
    x = x % p;
    while (y > 0):
         
        # If y is odd, multiply
        # x with result
        if (y & 1):
            res = (res * x) % p;
 
        # y must be even now
        y = y>>1; # y = y/2
        x = (x * x) % p;
     
    return res;

In [79]:

def miller_rabin(n, k):
    if n == 2:
        return True

    if n % 2 == 0:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for i in range(k):
        a = random.randrange(2, n - 1)
        x = power(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for j in range(r - 1):
            x = power(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True


In [80]:
def random_with_N_digits(n):
    range_start = 10**(n-1) #
    range_end = (10**n)-1 #exp(10,n) -1 #
    return randint(range_start, range_end)


In [84]:
random_with_N_digits(5)

18598

In [85]:
def generar_primo(l):
    while True:       
        a = random_with_N_digits(l) 
        if miller_rabin(a,100):
            return a

### Algoritmo extendido de euclides

https://www.geeksforgeeks.org/python-program-for-basic-and-extended-euclidean-algorithms-2/
    
https://www.techiedelight.com/extended-euclidean-algorithm-implementation/    

In [86]:
def gcdExtended(a, b): 
    # Caso base 
    if a == 0 :  
        return b,0,1
             
    gcd,x1,y1 = gcdExtended(b%a, a) 
     
    # Actualizo x y uso el resultado recursivo
    x = y1 - (b//a) * x1 
    y = x1 
     
    return gcd,x,y
      
def alg_ext_euclides(a,b):
    gcd = gcdExtended(a,b)
    return gcd[0]
 

### Inverso

https://www.geeksforgeeks.org/multiplicative-inverse-under-modulo-m/

In [87]:
def inverso(a, n):
     
    for x in range(1, n):
        if (((a%n) * (x%n)) % n == 1):
            return x
    return -1

### Claves Publicas y Privadas

Revisar lo de los largos y randoms

In [88]:
def generar_claves(l):
    l_ = random.randint(2, l)
    P = generar_primo(l) # l o numero random <= a l ?
    Q = generar_primo(l)
    N = P*Q
    phi_N = (P-1)*(Q-1)
    
    primo = 0 #Para iniciar la variable
    busca_primo_rel = True
    while busca_primo_rel:
        largo = l
        primo = generar_primo(random.randint(2, l*2)) #El primo relativo al azar debe ser de que largo? o puede ser cualquiera?
        if alg_ext_euclides(primo,phi_N) == 1:
            busca_primo_rel= False
    inverso_primo = inverso(primo, phi_N)
    P = (inverso_primo, N)
    S = (primo, N)
    
    #Escritura de archivo clave secreta
    fs = open('private_key', 'w')
    fs.write(str(P[0])+ "\n")
    fs.write(str(P[1])+ "\n")
    fs.close()
    
     #Escritura de archivo clave publica
    fp = open('public_key', 'w')
    fp.write(str(S[0])+ "\n")
    fp.write(str(S[1])+ "\n")
    
    return  [P, S]
            

In [93]:
#generar_claves(5)

[(2531032563, 7590722137), (8569699, 7590722137)]

### Cifrado y Decifrado

In [89]:
def enc(m):
    p = open("public_key", "r").read().splitlines()
    return pow(m, int(p[0]), int(p[1]))

In [90]:
def dec(m):
    s = open("private_key", "r").read().splitlines()
    return pow(m, int(s[0]), int(s[1]))